In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp500.csv')
#stocks

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#type(data)
#print(data) # Response = 200 means success
#print(data.status_code == 200)

In [5]:
print(data)

{'symbol': 'AMZN', 'companyName': 'Amazon.com Inc.', 'primaryExchange': 'AEASBSAENSA /ERLLTGQ) ONL G(TKMDC', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'iocaflfi', 'close': None, 'closeTime': None, 'closeSource': 'fiafcilo', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 3427.19, 'latestSource': 'IEX Last Trade', 'latestTime': 'April 21, 2021', 'latestUpdate': 1675293418879, 'latestVolume': None, 'iexRealtimePrice': 3512.31, 'iexRealtimeSize': 101, 'iexLastUpdated': 1678717192834, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 3410, 'previousVolume': 2699702, 'change': 28.2, 'changePercent': 0.0084, 'volume': None, 'iexMarketPercent': 0.04231887713165134, 'iexVolume': 95175, 'avgTotalVolume': 3353409

In [6]:
price = data['latestPrice']
#print(price)
market_cap = data['marketCap']
#print(market_cap/1000000000000)

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe = pd.DataFrame([[0,0,0,0]],columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
    index=my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AMZN,3427.19,1742643344737,N/A


Looping through the tickers in the list

In [9]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:10]:
#for stock in stocks['Ticker']:
    #print(stock)
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index=my_columns),
        ignore_index=True
        )

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,134.27,41074222939,N/A
1,AAL,21.00,13598385122,N/A
2,AAP,205.81,13602458484,N/A
3,AAPL,134.88,2276979254598,N/A
4,ABBV,115.90,197779111851,N/A
5,ABC,123.61,24866789163,N/A
6,ABMD,346.16,15597911827,N/A
7,ABT,122.80,227529956691,N/A
8,ACN,295.61,193660046246,N/A
9,ADBE,524.49,255042895642,N/A


Using the Batch API Calls to improve the performance. <br>
IEX Cloud limits the batch API calls to 100 tickers per request.

In [25]:
def chunks(lst, n):
    # Yield successive n-sized chunks from lst.
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [27]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
#symbol_groups
symbol_strings = []
for i in range(0, len(symbol_groups)):
    #print(i)
    #print(symbol_groups[i])
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    #print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,135.56,41964673131,N/A
1,AAL,22.00,13804960687,N/A
2,AAP,208.17,13555100496,N/A
3,AAPL,134.11,2324135822970,N/A
4,ABBV,115.50,202680107755,N/A
...,...,...,...,...
500,YUM,121.12,36680271520,N/A
501,ZBH,183.11,36820065598,N/A
502,ZBRA,495.68,27523476560,N/A
503,ZION,55.16,8843751998,N/A
